In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
from tqdm.autonotebook import tqdm
import os
import cftime
import json
from dask import array

from xhistogram.xarray import histogram

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
from dask.distributed import Client

client = Client("tcp://10.48.92.56:46261")
client

Client Scheduler: tcp://10.48.92.56:46261 Dashboard: /user/pangeo-data-pan--cmip6-examples-njjsfcc7/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
gcs = gcsfs.GCSFileSystem(token='anon')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrss,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN


In [4]:
dfs = pd.read_csv('pangeo.csv')

In [5]:
# file_attrs = json.load(open('file_attrs.txt','r'))
# all_attrs = set(['_'.join([a['source_id'],a['member_id'],a['experiment_id'],a['table_id'],a['variable_id']]) for a in file_attrs])

# def want(s, m, e, t, v):
#     key = s+'_'+m+'_'+e+'_'+t+'_'+v
#     return s+'_'+m+'_'+e+'_'+t+'_'+v

# dfs = df[df[['source_id','member_id','experiment_id','table_id','variable_id']].apply(lambda x: want(*x) in all_attrs, axis=1)]

# pangeo_attrs = set(dfs[['source_id','member_id','experiment_id','table_id','variable_id']].apply(lambda x: want(*x), axis=1).values)
# manual_attrs = [a for a in all_attrs if a not in pangeo_attrs]
# json.dump(list(pangeo_attrs), open('pangeo_loads.txt', 'w'))
# json.dump(list(manual_attrs), open('manual_loads.txt', 'w'))

# dfs.to_csv('pangeo_loads.csv')

In [6]:
def load_srch_data(df, source_id, expt_id):

    uri = df[(df.source_id == source_id) &
                         (df.experiment_id == expt_id)].zstore.values[0]
    
    ds = xr.open_zarr(gcs.get_mapper(uri), consolidated=True)
    return ds

def load_data(series):
    ds = xr.open_zarr(gcs.get_mapper(series.zstore), consolidated=True)
    return ds

def get_dims(ds):
    ds_coords = [l for l in list(ds.coords.keys()) if 'bnds' not in l and 'vert' not in l]
    dims = [[l for l in ds_coords if 'lat' in l][0], [l for l in ds_coords if 'lon' in l][0]]
    lon = ds.coords.get(dims[0])
    lat = ds.coords.get(dims[1])
    return lat, lon

def get_area(ds, df):
    var = ds.get(ds.variable_id)
    realm = ds.table_id[0].lower()
    area = None
    lat, lon = get_dims(ds)
    dims = [lat.name, lon.name]

    if len(lat) > 1000:
        return lat, "ncells"
    
    time, lon, area = np.meshgrid(ds.time, lon, np.cos(lat), indexing='ij')

    if isinstance(area, type(None)):
        df_area = df.query("variable_id == 'areacell"+realm+"' & source_id == '"+ds.source_id+"'")
        if len(df_area.zstore.values) == 0:
            print('failed area search: ', ds.source_id)
            return None
        ds_area = xr.open_zarr(gcs.get_mapper(df_area.zstore.values[0]), consolidated=True)
        area = ds_area.get("areacell"+realm)

    return area, dims

def avg_var(ds, df):
    area, dims = get_area(ds, df)
    var = ds.get(ds.variable_id)

    total_area = area.sum()
    
    ta_timeseries = (var * area).sum(dim=dims) / total_area
    
    if isinstance(ta_timeseries, type(None)):
        print('failed')
    return ta_timeseries



In [7]:
results = {}

In [8]:
for num in tqdm(list(range(len(dfs[4:])))):
#     try:
    s = dfs.iloc[num]
    name = '_'.join([s.source_id, s.experiment_id, s.member_id, s.variable_id])

    if name + '.npy' in os.listdir('data'):
        print(name)
        continue
        
    ds = load_data(s)
    if len(get_dims(ds)[0]) > 1000:
        continue
    m = None
    m = avg_var(ds, df)

    if not isinstance(m, type(None)):
#             results[name] = np.array([m.values, np.array([np.datetime64(t) for t in m.time.values])])
        np.save('data/'+name, np.array([m.values[:], np.array([np.datetime64(t) for t in m.time.values])[:]]))
#     except:
#         print(num)
#         continue

AWI-CM-1-1-MR_1pctCO2_r1i1p1f1_rlut
AWI-CM-1-1-MR_1pctCO2_r1i1p1f1_rsut
AWI-CM-1-1-MR_1pctCO2_r1i1p1f1_ts


KeyboardInterrupt: 

In [ ]:
lats = []
lons = []
for r in tqdm(range(len(dfs))):
    ds = load_data(dfs.iloc[r])
    ds_coords = [l for l in list(ds.coords.keys()) if 'bnds' not in l and 'vert' not in l]
    lat_key = [l for l in ds_coords if 'lat' in l]
    lon_key = [l for l in ds_coords if 'lon' in l]
    lats += [lat_key]
    lons += [lon_key]
print(set(lats))
print(set(lons))

In [ ]:
def percentiles(ds, df):
    Area = get_area(ds, df)
    Var = ds.get(ds.variable_id)
    
    percentiles = []
    
    for t in range(len(Var.time)):
        area = Area[t]
        var = Var[t]
        time = Var.time[t]
        
        weights = np.reshape(area.data, (np.prod(np.shape(var)),1))
        vals = np.reshape(var.data, (np.prod(np.shape(var)),1))
        weights = np.array(weights[~np.isnan(vals)])
        vals = np.array(vals[~np.isnan(vals)])

        idx = np.argsort(vals)
        vals = np.take_along_axis(vals, idx, axis=0)
        sorted_weights = np.array(np.take_along_axis(weights, idx, axis=0))
        total = np.nansum(weights)

        i = 0
        low = 0
        N = len(weights)
        pcts = np.array([0.05, 0.17, 0.5, 0.83, 0.95])
        pct_vals = []
        for j in range(0,len(vals)):
            low = low + sorted_weights[j]
            high = low + sorted_weights[min(j+1,N)]
            # If the cumulative weights are nearest the next percentile
            # Then mark down the value
            if low/total < pcts[i] and high/total >= pcts[i]:
                pct_vals += [[pcts[i], vals[j]]]
                if i == np.shape(pcts)[0] - 1:
                    break
                i = i + 1
                
        percentiles += [time, pct_vals]
    
    return percentiles
